In [4]:
import torch
import aeroslm
from huggingface_hub import hf_hub_download

pt_path = hf_hub_download(
    repo_id="samratkar/slm_tinystories",
    filename="slm_tinystories.pt",
)
# Load the model
model = aeroslm.GPT()
state_dict = torch.load(pt_path, map_location=torch.device('cuda'))
model.load_state_dict(state_dict)
model.eval()  # Set to evaluation mode

GPT(
  (transformer): ModuleDict(
    (wte): Embedding(50257, 384)
    (wpe): Embedding(128, 384)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-5): 6 x Block(
        (ln1): LayerNorm()
        (attn): CausalSelfAttention(
          (c_attn): Linear(in_features=384, out_features=1152, bias=True)
          (c_proj): Linear(in_features=384, out_features=384, bias=True)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln2): LayerNorm()
        (mlp): MLP(
          (c_fc): Linear(in_features=384, out_features=1536, bias=True)
          (gelu): GELU(approximate='none')
          (c_proj): Linear(in_features=1536, out_features=384, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm()
  )
  (lm_head): Linear(in_features=384, out_features=50257, bias=False)
)

In [5]:
model.infer("Once upon a time there was a pumpkin.")

'Once upon a time there was a pumpkin. It had grew out of the lake with some fish-colored strawberries and carrots. Bbleed around the leaf, the pumpkin spread its wings and ribbons in the sun. It was even less colorful!\n\nBecky was so happy that she had played with the picnic and the flag. She had so much fun playing with the pie!\n\nBut later, celebrate her mother was ready to go to sleep. She said it was time to sit down next bed and slip and hide carefully. She was ready to wash the pumpkin and the whale gave it to herself.\n\nMeanwhile, Bubora\'s mother heard her calling out. "Mom, Mom, what is that?"\n\nMary pointed to a man. "Mom, look at my Splash!"\n\nMom ran to her and saw the tomatoes. He was so excited to see her! She thanked her mum for helping her daughter.\n\nThe mile lit up with the cheese and smiles a smile. "I was a cool'

In [6]:
model.infer("A little girl went to the woods")

"A little girl went to the woods. The girl noticed a worm and decided to have it a nut and cut it with a sharp bark, and he was very honest. So, she apologized to her mom and daddy for being kind to him. \n\nThat day, the dove flew over and jumped up and down. He was so happy that he had such a great chance to do the magic worm.Once upon a time, there was a little girl named Lily. She loved to play outside in the sun. One night, her mommy told her that she needed to go home and take a walk across the swings. Lily was so excited, she forgot about all the next jog. \n\nWhile playing, she was driving, she saw something shiny on the ground! She searched but the jacket was pink and pretty. Then, she saw a park. She looked and he couldn't buy it! \n\nAs she was running after a while, her mom called him to her for lunch. She gave them two"

In [7]:
model.infer("The pilot was flying a 747-8 in cruise, when there was a huge turbulence!")

"The pilot was flying a 747-8 in cruise, when there was a huge turbulence! He was so excited and waved, then he glued off his steps. Everywhere he. though, everyone jumped up and down the mountain felt the horizon! They laughed and laughed as they danced around and with the ship improving hands. \n\nThe locked harbor, they and the reindeer were amazed by the whole joy. They watched and enjoy each time they finally spent the journey. The family were happy and the map was so safe!One day a rabbit were walking in the woods. They were walking, but they couldn't see anything floated even more. Suddenly, the rabbit stopped to see a little squirrel. The duck was! Tom was a slow, but he kept jumping and having fun. \n\nHe hopped faster and faster, until he finished noon. He had hid by a shortcut. Then one of the bunny followed them.\n\nHe stopped and watched the squirrel go, crying. Tom said hello to the squirrel and not accident him. The forest pinched down follows the squirrel"